In [10]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sklearn.metrics as metrics
from keras import models
from keras import layers

In [11]:
#!pip install keras
#!pip install tensorflow
#!pip install scikeras

In [12]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [13]:
x = np.arange(10).reshape(-1, 1)
y = np.array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1.])

In [14]:
print(x)
print(y)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]
[0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]


In [15]:
import holoviews as hv
import panel as pn
import hvplot.pandas
hv.extension('bokeh')

In [16]:
plot_val = hv.Scatter(data=(x,y), label='Исходные значения')
plot_val.opts(marker='circle',size=10, alpha=1, color='green',
              show_grid = True, xlabel='X', ylabel='Y',
              height=500, width=600, 
              title='Обучение')
plot_val

:Scatter   [x]   (y)

In [17]:
model = models.Sequential()
model.add(layers.Dense(10,kernel_initializer='RandomNormal', activation='sigmoid', input_shape=(1,)))
model.add(layers.Dense(2,kernel_initializer='RandomNormal', activation='sigmoid'))
model.add(layers.Dense(1,kernel_initializer='RandomNormal', activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=[tf.keras.metrics.MeanSquaredError()])

In [18]:
import os
model_checkpoint = tf.keras.callbacks.ModelCheckpoint (os.path.join('checkpoints','checkpoint-{epoch}.keras'))
model.fit(x, y, epochs=100, batch_size=1, verbose=2,  callbacks = [model_checkpoint])

Epoch 1/100
10/10 - 1s - 51ms/step - loss: 0.2523 - mean_squared_error: 0.2523
Epoch 2/100
10/10 - 0s - 3ms/step - loss: 0.2517 - mean_squared_error: 0.2517
Epoch 3/100
10/10 - 0s - 3ms/step - loss: 0.2514 - mean_squared_error: 0.2514
Epoch 4/100
10/10 - 0s - 3ms/step - loss: 0.2511 - mean_squared_error: 0.2511
Epoch 5/100
10/10 - 0s - 3ms/step - loss: 0.2509 - mean_squared_error: 0.2509
Epoch 6/100
10/10 - 0s - 3ms/step - loss: 0.2507 - mean_squared_error: 0.2507
Epoch 7/100
10/10 - 0s - 3ms/step - loss: 0.2505 - mean_squared_error: 0.2505
Epoch 8/100
10/10 - 0s - 4ms/step - loss: 0.2503 - mean_squared_error: 0.2503
Epoch 9/100
10/10 - 0s - 3ms/step - loss: 0.2500 - mean_squared_error: 0.2500
Epoch 10/100
10/10 - 0s - 4ms/step - loss: 0.2498 - mean_squared_error: 0.2498
Epoch 11/100
10/10 - 0s - 4ms/step - loss: 0.2496 - mean_squared_error: 0.2496
Epoch 12/100
10/10 - 0s - 4ms/step - loss: 0.2494 - mean_squared_error: 0.2494
Epoch 13/100
10/10 - 0s - 4ms/step - loss: 0.2493 - mean_squ

In [21]:
frame = 100
model.load_weights('./checkpoints/checkpoint-'+str(frame)+'.keras')
z = model.predict(x)
print(z)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
[[0.56853104]
 [0.5702016 ]
 [0.5717593 ]
 [0.5731419 ]
 [0.5743196 ]
 [0.57528895]
 [0.5760664 ]
 [0.5766781 ]
 [0.5771556 ]
 [0.577521  ]]


In [22]:
results = model.evaluate(x, y)
print('test mae, test mse:', results)
score = metrics.r2_score(y, z)
print('score:', score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 0.2382 - mean_squared_error: 0.2382
test mae, test mse: [0.2381548136472702, 0.2381548136472702]
score: 0.0076883465598190925


In [23]:
plot_nn = hv.Scatter(data=(x,z), label='Предсказанные значения')
results = model.evaluate(x, y, verbose=0)
plot_nn.opts(marker='square',size=10, alpha=0.5, color='red',
             show_grid = True, xlabel='X', ylabel='Z',
             height=500, width=600)
overlay = hv.Overlay(plot_val+plot_nn).opts(title='Предсказание - score:%.5f, test mae:%.5f, test mse:%.5f' %(score, results[0],results[1]), 
                                             legend_position='top')
overlay

:Overlay
   .Scatter.Исходные_значения      :Scatter   [x]   (y)
   .Scatter.Предсказанные_значения :Scatter   [x]   (y)

In [25]:
n_epoch = pn.widgets.IntSlider(name='Эпоха', start=1, end=5000, value=1, orientation='horizontal', min_width=600 )

def get_prediction(frame):
    model.load_weights('.\\checkpoints\\checkpoint-'+str(frame)+'.keras')
    z = model.predict(x, verbose=0)
    plot_c_nn = hv.Curve(data=(x,z), label='Предсказанные значения')
    plot_c_nn.opts(line_width=3, alpha=0.5, color='black',
                   line_dash=[10,2],
                   show_grid = True, xlabel='X', ylabel='Z',
                   height=500, width=600)

    plot_s_nn = hv.Scatter(data=(x,z), label='Предсказанные значения')
    plot_s_nn.opts(marker='square',size=10, alpha=0.5, color='red',
             show_grid = True, xlabel='X', ylabel='Z',
             height=500, width=600)
    results = model.evaluate(x, y, verbose=0)
    score = metrics.r2_score(y, z)
    overlay = hv.Overlay(plot_val+plot_c_nn+plot_s_nn).opts(title='Предсказание - score:%.5f, test mae:%.5f, test mse:%.5f' %(score, results[0],results[1]),
                                                            legend_position='top')
    return (overlay)


pn.Column(
    pn.pane.HoloViews(
        pn.bind(get_prediction, 100)
    ).servable(),
   100
)

ValueError: HoloViews pane does not support objects of type 'function'.

In [26]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 10)             │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │            22 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 92 (372.00 B)

 Trainable params: 45 (180.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 47 (192.00 B)

In [ ]:
#!pip install keras-visualizer

In [27]:
%matplotlib inline 
from keras_visualizer import visualizer
visualizer(model, file_name='graph', file_format='png') # save model
import matplotlib.image as mpimg
plt.subplots(figsize=(8,6))
plt.imshow(mpimg.imread('graph.png'))

ValueError: [Keras Visualizer] Error while visualizing: <class 'Exception'>

: 

In [26]:
for i,layer in enumerate(model.layers):
    weights = layer.get_weights()[0]
    print('layer %i weights: \n' %(i), weights)

layer 0 weights: 
 [[ 0.79686606 -0.8251527  -0.84571123  0.7951303  -0.8397736   0.8080469
  -0.8423161   0.80834377  0.7959168   0.7583948 ]]
layer 1 weights: 
 [[ 1.4808154  1.4155157]
 [-3.4560862 -3.449868 ]
 [-3.5806956 -3.6130044]
 [ 1.3837703  1.471519 ]
 [-3.575654  -3.5820873]
 [ 1.5141081  1.5460136]
 [-3.5680273 -3.5857468]
 [ 1.5859364  1.4895356]
 [ 1.3804241  1.5201352]
 [ 1.2560953  1.1836398]]
layer 2 weights: 
 [[6.4169364]
 [6.4914403]]


In [27]:
plot_w=[]
for i,layer in enumerate(model.layers):
    weights = layer.get_weights()[0]
    shape = weights.shape
    plot_w.append( hv.Raster(weights).opts(title ='Слой %i' %(i), cmap='hot',colorbar=True, show_frame=True, xaxis=False, yaxis=False, height=300, width=300))
layout=hv.Layout(plot_w)
layout.opts( title ="Веса", width=600, height=500)

:Layout
   .Raster.I   :Raster   [x,y]   (z)
   .Raster.II  :Raster   [x,y]   (z)
   .Raster.III :Raster   [x,y]   (z)

In [ ]:
import bokeh.io
bokeh.io.output_notebook()